<a href="https://colab.research.google.com/github/Natan1995/Algoritmos_De_Optimizacion/blob/main/Trabajo_Practico/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Gary Córdova<br>
Url: https://github.com/Natan1995/Algoritmos_De_Optimizacion/tree/main/Trabajo_Practico<br>
Problema: Organizar los horarios de partidos de La Liga<br>

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.





                                        

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sympy import symbols, diff

import random
import itertools
import math
import warnings

In [ ]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

In [ ]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",200)
pd.set_option("expand_frame_repr", True)
pd.set_option("max_info_columns", 200)
pd.set_option("display.float_format", '{:.2f}'.format )
#pd.set_option("precision", 2)

In [ ]:
warnings.filterwarnings('ignore')

# 1. Espacio de soluciones del problema

*   ¿Cuantas posibilidades hay sin tener en cuenta las restricciones? (*)<br>
*   ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.


  




Para obtener el espacio de soluciones debemos seguir 3 fases:

1.   Tenemos 20 equipos y debemos agruparlos en grupos de 2 para obtener todas las posibilidades de enfrentamientos; es decir tenemos $\binom{20}{2}$ soluciones. Esto nos da un total de 190 posibles combinaciones.

2.   Ahora tenemos que distribuir esos 190 soluciones en 16 distintos horarios; es decir, tenemos un espacio de soluciones de  ${190x16}$ combinaciones (3040 combinaciones).

3.   Finalmente Ahora tenemos que distribuir esas soluciones en grupos de 10; esto es debido a que cada jornada tiene 10 enfrentamientos entre ellos. Para lograr este objetivo analizamos la cantidad de grupos de 10 en 10 que se pueden generar con la solución de la fase 2, para eso usamos una comnbinatoria de $\binom{3040}{10}$,


Por otro lado, el problema tiene ciertas casuisticas que vamos a considerar aqui:

1.   De los 20 equipos, los equipos clase C no se pueden enfrentar entre ellos.; es decir que al total de combinaciones le restaremos las combinaciones de los equipos clase C, calculamos ($\binom{20}{2}$ - $\binom{6}{2}$), esto nos da un total de 175 posibles combinaciones.

2.   Ahora tenemos que distribuir esos 175 soluciones en 10 distintos horarios (dado que tienen un ratio positivo en la tabla de horarios de la liga); es decir tenemos ${175x10}$ combinaciones (1750 combinaciones).

3.   Finalmente Ahora tenemos que distribuir esas soluciones en grupos de 8; debido a que tenemos 2 horarios (Lunes y Viernes) que son fijos. Por ende debemos generar grupos de 8 en 8 que se pueden generar con la solución de la fase 2, para eso usamos una comnbinatoria de $\binom{1750}{8}$,



---




# 2. Modelo para el espacio de soluciones <br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Para este problema hemos decidido aplicar listas para definir los 20 equipos que tendra la liga. Adicional a ello usaremos matrices para definir los horarios y los ingresos generados por la combinación de los equipos. Para ellos manejaremos la siguiente nomenclatura:






1.   Para las categorias de equipos:

>*   Categoria A => 0
>*   Categoria B => 1
>*   Categoria C => 2

2.   Las siguientes nomenclaturas para los horarios:

>*   12 horas => 0
>*   16 horas => 1
>*   18 horas => 2
>*   20 horas => 3

3.   Las siguientes nomenclaturas para los días:

>*   Viernes => 0
>*   Sabado => 1
>*   Domingo => 2
>*   Lunes => 3




Ahora que hemos definido la nomenclatura a utilizar, generaremos el código:

In [ ]:
arr_equipos = ['Real Madrid','Barcelona','Atletico Madrid','Real Betis','Granada','Athletic Club de Bilbao','Getafe','Villareal','Real Sociedad','Osasuna','Valencia','Celta de Vigo','Sevilla','Girona',
               'Deportivo Alaves','Cádiz','Rayo Vallecano','Mallorca','UD las Palmas','Almeria']
cat_equipos = ['A','A','A','B','B','B','B','B','B','B','B','B','B','B','C','C','C','C','C','C']

data_liga  = {
    'Equipo': arr_equipos,
    'Categoría': cat_equipos
}

In [ ]:
matriz_audiencia = np.array([[0, 0.55, 0.45,0],[0, 0.7, 0.75,0],[0, 0.80, 0.85,0],[0.4, 1, 1,0.4] ])
matriz_ingresos = np.array([[2, 1.3, 1], [1.3, 0.9, 0.75]])
arr_coincidencias = [0,0.25,0.45,0.60,0.70,0.75,0.78,0.80,0.80]
arr_equipos = [0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2]

Generaremos algunos dataframes usando la libreria pandas para poder visualizar como quedaria nuestras estructuras de datos:

In [ ]:
df_liga = pd.DataFrame(data_liga)
df_ingresos = pd.DataFrame(matriz_ingresos, index=['A', 'B'], columns=['A', 'B', 'C'])
df_audiencias = pd.DataFrame(matriz_audiencia, index=['12h', '16h','18h','20h'], columns=['Viernes', 'Sabado', 'Domingo','Lunes'])

In [ ]:
# Ingresos generados(en millones) por enfrentamientos entre equipos
df_ingresos

,A,B,C
A,2.00,1.30,1.00
B,1.30,0.90,0.75


In [ ]:
#Ponderación que se le aplica a los ingresos dependiendo del horario y las horas
df_audiencias

,Viernes,Sabado,Domingo,Lunes
12h,0.00,0.55,0.45,0.00
16h,0.00,0.70,0.75,0.00
18h,0.00,0.80,0.85,0.00
20h,0.40,1.00,1.00,0.40


In [ ]:
# Distribución de los categorias de la liga segun su número de seguidores
df_liga.groupby('Categoría').count().reset_index()

,Categoría,Equipo
0,A,3
1,B,11
2,C,6


Finalmente el resultado final del ejercicio sera encontrar un arreglo de tuplas cuya combinación maximice los ingresos recaudados por la liga.
Por ejemplo si tuvieramos 2 partidos, sería: [(0,1,[0,2]),(1,2,[1,2])].
Esto podría leerse de la siguiente manera:

*   El primer partido seria entre uno de clasificación A vs uno de clasificación B el día Domingo a las 12h.
*   El segundo partido seria entre uno de clasificación B vs uno de clasificación C el día Domingo a las 16h.






---



# 3. Modelo para el espacio de soluciones <br>


*   (*)¿Cual es la función objetivo?
*   (*)¿Es un problema de maximización o minimización?

El problema dado es un problema de maximización; debido a que su objetivo es maximizar los ingresos percibidos por la liga; si bien es importante incrementar la audiencia, esta debe estar alineada a una buena distribución de los partidos para asi generar los ingresos deseados.
La formulación del problema puede ser dada de la siguiente forma:

Función Objetivo: Maximizar( $\sum_{i=1}^{n} (Partido_{e_{1},e_{2}} * Ponderacion_{d,h}*(1-correccion_{d,h}))$ )

<br> Dado las siguientes restricciones:

*   $\forall e_{1},e_{2} \in \{A,B,C\}$
*   $\forall d \in \{Viernes,Sabado,Domingo,Lunes\}$
*   $\forall h \in \{12h,16h,18h,20h\}$
*   $\exists (Partido_{e_{1},e_{2},d})$; en donde  $\forall d \in \{Viernes,Lunes\}$









---



# 4. Algoritmo por Fuerza Bruta

En esta sección, implementaremos un algoritmo de fuerza bruta que satisfaga nuestras restricciones y logre genererar una jornada para la liga de futbol

In [ ]:
# Funcion para definir los enfrentamientos validos
def get_enfrent_validos(matriz_ingresos):

  filas, columnas = matriz_ingresos.shape
  indices_filas = list(range(filas))
  indices_columnas = list(range(columnas))

  enfr_validos = list(itertools.product(indices_filas, indices_columnas)) +  list(itertools.product(indices_columnas, indices_filas))
  return list(set(enfr_validos))

In [ ]:
# Funcion para definir los horarios en que se puede jugar un partido
def get_horar_validos(matriz_audiencias):

  filas, columnas = matriz_audiencias.shape
  indices_filas = list(range(filas))
  indices_columnas = list(range(columnas))

  #hor_validos = list(itertools.product(indices_filas, indices_columnas))
  #hor_validos = [list(comb) for comb in itertools.product(indices_filas, indices_columnas)]
  hor_validos = [list(comb) for comb in itertools.product(indices_filas, indices_columnas) if matriz_audiencias[comb[0]][comb[1]] > 0]
  return hor_validos

In [ ]:
# Funcion que genera emparejamientos de equipos
def get_emp_equipos(arr_equipos, hor_oblig):

  # Iterar a través de los equipos disponibles
  emparejamientos = []
  equipos_disponibles = arr_equipos.copy()
  enfr_validos = get_enfrent_validos(matriz_ingresos)
  hor_validos = get_horar_validos(matriz_audiencia)
  horario_jornada = np.zeros((matriz_audiencia.shape))
  cont = 0

  while equipos_disponibles:


      #equipo_1 = equipos_disponibles.pop()
      equipo_1 = random.choice(equipos_disponibles)
      equipos_disponibles.remove(equipo_1)

      # Eliminar el equipo asignado de los equipos disponibles
      equipo_2 = random.choice(equipos_disponibles)
      equipos_disponibles.remove(equipo_2)

      equipo_local = min(equipo_1,equipo_2)
      equipo_rival = max(equipo_1,equipo_2)

      # Verificar si el emparejamiento está en enfrentamientos_validos
      if(equipo_local, equipo_rival) in enfr_validos:

        if (cont < 2):
          horario_valido = random.choice(hor_oblig)
          hor_oblig.remove(horario_valido)
        else:
          horario_valido = random.choice(hor_validos)

        # Agregar el emparejamiento a la lista
        emparejamientos.append((equipo_local, equipo_rival, horario_valido))
        horario_jornada [horario_valido[0],horario_valido[1]] = horario_jornada [horario_valido[0],horario_valido[1]] + 1
        cont+=1

  return emparejamientos, horario_jornada

In [ ]:
# Esta función nos permite obtener el costo percibido de un enfrentamiento, tomando en consideración el horario y la clasificación de los equipos
def get_costo_jornada(enfr_equipos,horario_jornada):

  ingres_tot_aud = 0

  if(enfr_equipos is not None):
    for emparejamiento in enfr_equipos:

        equipo_local, equipo_rival, horario_valido = emparejamiento

        base_ingresos = matriz_ingresos[equipo_local,equipo_rival]
        ratio_horario = matriz_audiencia[horario_valido[0],horario_valido[1]]
        correc_coinc_jornada = arr_coincidencias[int(horario_jornada[horario_valido[0],horario_valido[1]])]
        ingreso_percibido = (base_ingresos * ratio_horario) * (1-correc_coinc_jornada)

        ingres_tot_aud = round(ingres_tot_aud + ingreso_percibido,2)

  return ingres_tot_aud

In [ ]:
#Ejemplo de uso para calcular los ingresos de una jornada y como quedaria la liga
enfr_equipos,horario_jornada = get_emp_equipos(arr_equipos, [[3,0],[3,3]])
ingres_tot_aud = get_costo_jornada(enfr_equipos,horario_jornada)

print(f"Enfrentamientos: {enfr_equipos}")
print(f"Ingreso Recibido: {ingres_tot_aud}")

Enfrentamientos: [(1, 1, [3, 0]), (1, 2, [3, 3]), (0, 1, [0, 2]), (0, 1, [3, 1]), (1, 2, [2, 1]), (1, 2, [3, 2]), (1, 1, [3, 0]), (0, 1, [3, 0]), (1, 2, [3, 2])]
Ingreso Recibido: 3.4


Vamos a implementar nuestro código para generar 100 simulaciones y quedarnos con la jornada de futbol que mejor costo recaude:

In [ ]:
#Realizaremos una simulacion para generar 10 posibles jornadas
n_simulaciones = 10
#n_simulaciones = get_esp_sol(arr_equipos,16,10)
i = 0
mejor_simulacion = []
mejor_ingreso = 0
jornadas_simuladas = []

while(i < n_simulaciones):

  #Agregamos una variable con los horarios que deben ser obligatorios
  #[3,0] y [3,3] indican que todas las jornadas de futbol deben tener algun partido en Lunes y Viernes
  enfr_equipos,horario_jornada = get_emp_equipos(arr_equipos,[[3,0],[3,3]])

  if(enfr_equipos not in jornadas_simuladas):
    ingres_tot_aud = get_costo_jornada(enfr_equipos,horario_jornada)
    jornadas_simuladas.append(enfr_equipos)

    if(ingres_tot_aud > mejor_ingreso):
      mejor_ingreso = ingres_tot_aud
      mejor_simulacion = enfr_equipos

      i = i +1

print(f"Jornada Definida: {mejor_simulacion}")
print(f"Ingreso Maximo: {mejor_ingreso}")

Jornada Definida: [(1, 2, [3, 0]), (1, 2, [3, 3]), (0, 0, [3, 2]), (1, 1, [2, 1]), (1, 1, [3, 1]), (1, 2, [0, 2]), (0, 1, [2, 2]), (1, 2, [3, 1]), (1, 1, [1, 2])]
Ingreso Maximo: 5.0




---



# Complejidad del algoritmo - Fuerza Bruta

Se podría considerar la solución anterior como una solución aleatoria; dado que consideramos el número de simulaciones de manera arbitraria, no existe un aval de que este número de soluciones nos de buenos resultados.

Por otro lado, de querer obtener mejores resultados, tendriamos que aumentar el número de soluciones; esto no es nada optimo dado que tenemos un espacio de soluciones extremadamente alto.


Analizando el código, tenemos:

1.   La complejidad del bucle depende del "número de simulaciones a implementar"; esto nos daría una complejidad de O(m), siendo m el número de simulaciones.
2.   En cada bucle de las simulaciones se hace un llamado a la función "get_emp_equipo" que tiene una complejidad en el peor de los casos de O(n/2) u O(n), siendo n el número de equipos que tendremos en nuestra liga.
3.  El resto de funciones a implementar son de complejidad constante, por ende podriamos indicar que la complejidad del algoritmo sería de O(m*n), donde m es el número de simulaciones y n es el número de equipos.





---



# Mejora sobre el algoritmo de Fuerza Bruta
Diseño un algoritmo que mejore la complejidad del algortimo por fuerza bruta(*)

El algoritmo implementado anteriormente es un algoritmo de fuerza bruta que utiliza un limite de iteraciones debido a que este consumiria todos los recursos disponibles y no daría una solución al respecto.

Debido a ello, para mejorar la solución a este problema haremos uso de la tecnica Tabu Search que a nuestro parecer se podría adaptar bastante bien a este problema por estos motivos:

1.   Podriamos reutilizar el código para obtener uan solución inicial, usaremos la función "get_emp_equipos" para obtener una distribución de equipos de una jornada inicial.
2.   Utilizariamos la exploración de vecindad para obtener vecinos a nuestra solución inicial que tenga un costo de recaudación mayor a nuestra solución inicial.

3.  Otras de las ventajas de la tecnica a utilizar sería la facilidad de implementación y flexibilidad de esta tecnica a distintos problemas.

4. Finalmente, consideramos que es una tecnica bastante robusta, que se adapta a diversos problemas de distintos ambitos.



In [ ]:
# Definimos una funcion que nos permitiria generar vecinos de nuestra solucion input
def generar_vecindario(jornada):
  vecindario = []
  for i in range(0,len(jornada)-1):
    for j in range(i+1, len(jornada)):
      vecina = jornada.copy()
      int_1 = (jornada[i][0],jornada[i][1],jornada[j][2])
      int_2 = (jornada[j][0],jornada[j][1],jornada[i][2])
      vecina[i] = int_1
      vecina[j] = int_2
      vecindario.append(vecina)
  return vecindario

In [ ]:
#Implementamos una función de apoyo que nos permitira obtener el horario de la jornada vecina generada
def get_jornada(jornada):
  hor_validos = get_horar_validos(matriz_audiencia)
  horario_jornada = np.zeros((matriz_audiencia.shape))

  for i in range(len(jornada)):
    horario_jornada[enfr_equipos[i][2][0], enfr_equipos[i][2][1]] = horario_jornada[enfr_equipos[i][2][0], enfr_equipos[i][2][1]]  + 1

  return horario_jornada

In [ ]:
# Obtenemos los enfrentamientos validos segun clasificacion de los equipos y los horarios disponibles en donde pueden enfrentarse
enfr_validos = get_enfrent_validos(matriz_ingresos)
hor_validos = get_horar_validos(matriz_audiencia)

In [ ]:
# Ejemplo de uso de la funcion para obtener una jornada de futbol aleatoria, con partidos obligatorios Lunes y Viernes
enfr_equipos,horario_jornada = get_emp_equipos(arr_equipos, [[3,0],[3,3]])
ingres_tot_aud = get_costo_jornada(enfr_equipos,horario_jornada)

print(f"Enfrentamientos: {enfr_equipos}")
print(f"Ingreso Recibido: {ingres_tot_aud}")

Enfrentamientos: [(1, 2, [3, 0]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [2, 1]), (1, 2, [2, 1]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Ingreso Recibido: 3.3


In [ ]:
# Ejemplo de uso de generación de vecindad de la jornada de liga aleatoria
vecindario = generar_vecindario(enfr_equipos)

# Listamos todos los vecinos que hemos generado
for i in range(len(vecindario)):
  print(f"Vecino N°{i}: {vecindario[i]}")

Vecino N°0: [(1, 2, [3, 3]), (0, 1, [3, 0]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [2, 1]), (1, 2, [2, 1]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Vecino N°1: [(1, 2, [3, 0]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [2, 1]), (1, 2, [2, 1]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Vecino N°2: [(1, 2, [0, 2]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [3, 0]), (1, 1, [2, 1]), (1, 2, [2, 1]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Vecino N°3: [(1, 2, [2, 1]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [3, 0]), (1, 2, [2, 1]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Vecino N°4: [(1, 2, [2, 1]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [2, 1]), (1, 2, [3, 0]), (0, 2, [3, 2]), (1, 2, [3, 2]), (0, 1, [0, 1]), (1, 1, [3, 3])]
Vecino N°5: [(1, 2, [3, 2]), (0, 1, [3, 3]), (1, 2, [3, 0]), (1, 2, [0, 2]), (1, 1, [2, 1]), (1, 2, [2, 1]), (0, 2, [3, 0]), (1, 2, [3,

In [ ]:
# Definimos nuestro metodo que hace uso de la tecnica Tabu Search
def tabu_search(arr_equipos,max_iter, tam_tabu):

  # Definimos nuestra jornada inicial aleatoria con partidos obligatorios Lunes y Viernes
  solucion_referencia,horario_referencia = get_emp_equipos(arr_equipos, [[3,0],[3,3]])
  mejor_solucion = solucion_referencia.copy()
  mejor_horario = horario_referencia.copy()
  tabu_list = []

  print(f"Ingreso Inicial: {get_costo_jornada(mejor_solucion, mejor_horario)}")

  N=0
  while N < max_iter:
    N+=1
    mejor_vecino = None
    mejor_horario = None
    mejor_ingreso = 0

    #Generamos un vecindario de nuestra jornada inicial
    vecindario = generar_vecindario(solucion_referencia)

    # Recorremos nuestro vecindario y obtenemos el vecino que genere el mayor ingreso
    for vecino in vecindario:
      horario_jornada_vecino = get_jornada(vecino)
      vecino_ingreso = get_costo_jornada(vecino, horario_jornada_vecino)
      if vecino_ingreso > mejor_ingreso and (vecino not in tabu_list or vecino_ingreso > get_costo_jornada(mejor_vecino, mejor_horario)):
        mejor_vecino = vecino
        mejor_ingreso = vecino_ingreso
        mejor_horario = horario_jornada_vecino

    if mejor_vecino is not None:
      solucion_referencia = mejor_vecino
      if mejor_ingreso > get_costo_jornada(mejor_solucion, mejor_horario):
          mejor_solucion = mejor_vecino

    # Añadimos a la lista tabu
    tabu_list.append(solucion_referencia)

    # Eliminamos de la lista tabu segun el tamaño predefinido de la lista tabu
    if len(tabu_list) > tam_tabu:
      tabu_list.pop(0)

  print(f"Ingreso Final: {get_costo_jornada(mejor_solucion, mejor_horario)}")

  return mejor_solucion, mejor_horario, get_costo_jornada(mejor_vecino, mejor_horario)

In [ ]:
# Ejemplo de uso de la función Tabu search
# Hacemos 10 mil iteraciones con un tamaño de lista Tabu de 15 soluciones
mejor_solucion, mejor_horario, mejor_ingreso = tabu_search(arr_equipos,10000, 15)
print(f"Enfrentamientos: {mejor_solucion}")

Ingreso Inicial: 3.97
Ingreso Final: 5.28
Enfrentamientos: [(0, 1, [2, 2]), (0, 2, [3, 3]), (1, 2, [3, 3]), (1, 2, [3, 0]), (0, 2, [2, 1]), (1, 2, [3, 0]), (0, 2, [0, 2]), (0, 0, [3, 1]), (0, 2, [3, 2])]
Ingreso Recibido: 5.28


En la aplicación lineas arriba podemos observar que la solución de jornada inicial aleatoria presenta un ingreso de 3.97 millones de soles de ingresos. Posteriormente a aplicar la tecnica Tabu Search, logramos mejorar los ingresos percibidos a 5.28 millones de soles.



---



# Complejidad del algoritmo - Tabu Search

Analizando el nuevo código, tenemos:

1. La complejidad del algoritmo depende de dos importantes factores, el número de iteraciones y la función de generación de vecindario.
2. El resto de funciones utilizadas son de complejidad constante.
3. La función de generación de vecindario recorre 2 veces el número de equipos  (2 for anidados), por lo que podriamos indicar que tiene complejidad O(n²).
4. Dado que la generación de vecindario se genera un número arbitrario de iteraciones, podriamos indicar que la complejidad del algoritmo es de O(n_iteraciones * n²)



---



# Ejecución del algoritmo Tabu Search

En esta sección se abarcaran 2 puntos:

1.   Diseño de datos aleatorios.
2.   Aplicación del algoritmo Tabu Search para los datos generados.



In [ ]:
# Función que genera n equipos de clasificacion A, B o C
# El número de equipos de cada clase se genera de manera aleatoria
def get_equipos_aleat(n):
  arr_equipos = [random.randint(0, 2) for _ in range(n)]
  return arr_equipos

In [ ]:
# Función que emula un juedo de datos aleatorios
# Definimos 25 equipos de la liga
arr_equipos_aleat = get_equipos_aleat(25)
print(arr_equipos_aleat)

# Uso de la tecnica de Tabu Search para encontrar la combinación de equipos que maximice los ingresos de audiencia
mejor_solucion, mejor_horario, mejor_ingreso = tabu_search(arr_equipos,10000, 15)

[0, 2, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 0, 0, 0, 1, 2, 2, 0]
Ingreso Inicial: 2.72
Ingreso Final: 4.45




---



# Conclusiones

Describir brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño:

La solución desarollada en este trabajo es una ejemplificación de como un algoritmo mas sofisticado puede mejorar de manera aceptable el rendimiento de un algoritmo de fuerza bruta; sin embargo existen algunos puntos que no considera y que podrian implementarse en futuros ejercicios para mejorar la calidad de la solución presentada.

Entre los principales puntos identificados, tenemos:

1.   Utilizar diferentes generadores de vecindarios.
2.   La asignación de horarios obligatorios que se desarollo no es la más eficiente (es algo arbitraria y permite cumplir con la restricción); se podría desarollar un algoritmo que cumpla la restricción y mejore incluso el rendimiento de la solución actual.
3.   Realizar mayor cantidad de simulaciones con diferentes horarios y matrices de audiencias.
4.   La implementación mostrada genera una jornada de una liga de futbol, pero sería interesante realizar todas las jornadas de una competencia (se tendria que evitar enfrentamientos repetidos y considerar dentro de la función la diferencia entre partidos de local y visitante).
5.   Probar otros algoritmos de busqueda local, como Recocido Simulado o Grasp. Asimismo, podriamos considerar el uso de algoritmos Bioinspirados como el colonia de hormigas (ACO) o el colonia de abejas (ABC).



# Exportar

In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!cp '/content/drive/MyDrive/Colab Notebooks/Seminario_Algoritmos_Final.ipynb' ./

In [13]:
!jupyter nbconvert --to PDF "Seminario_Algoritmos_Final.ipynb"

[NbConvertApp] Converting notebook Seminario_Algoritmos_Final.ipynb to PDF
[NbConvertApp] Writing 81466 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 89723 bytes to Seminario_Algoritmos_Final.pdf
